# **Examen final de Desarrollo de Aplicaciones para la Visualización de Datos**

Nombre: Yago

Apellidos: Tobio Souto

Tiempo: *2 horas y 30 minutos*

## **Contexto del ejercicio**

Un banco portugues desea entender en más detalle las campañas de marketing directo que ha realizado en los últimos meses a más de 40 mil clientes. Las campañas de marketing se basaron en llamadas telefónicas. Muchas veces era necesario más de un contacto con un mismo cliente, para saber si el producto (depósito bancario) sería ('sí') o no ('no') contratado.

El objetivo del análisis es buscar patrones para entender mejor que tipo de perfil tienen los clientes que han contratado el depósito para buscar en su base de datos otros clientes parecidos para aumentar la respuesta y el ROI de futuras campañas de marketing directo intentando vender el mismo depósito. Por lo tanto, se pide:

1.  Realizar un análisis descriptivo de los datos con al menos, 6 visualizaciones diferentes. (3 Puntos) (*)

2.  Montar un dashboard con al menos, 4 visualizaciones diferentes, que incluyan 2 componentes interactivas y 1 callback (5 Puntos) (*)

3. Concluir todo este análisis haciendo recomendaciones para la mejora de futuras campañas de contacto directo a partir de los resultados obtenidos de los análisis realizados con los datos. (2 Puntos)

Para realizar este análisis se provee de un juego de datos con las siguientes variables:

- age	- Edad (numérica)
- job - Ocupación (categórica)
- marital - Estado civil (categórica)
- education - Nivel educativo (categórica)
- default - ¿Tiene algún prestamo en default? (binaria)
- balance - Balance medio anual en euros (numérica)
- housing - ¿Tiene una hipoteca? (binaria)
- loan -  ¿Tiene un prestamo personal? (binaria)
- contact - Tipo de contacto (categórica)
- day_of_week - Último día de la semana que fue contactado (fecha)
- month - Último mes que fue contactado (fecha)
- duration - Duración de la última vez que fue contactado en segundos (entera)
- campaign - Número de veces que fue contactado (numérica)
- pdays - Número de días que pasaron después de que el cliente fue contactado por última vez desde una campaña anterior (numérica; -1 significa que el cliente no fue contactado previamente)
- previous - Número de contactos realizados durante esta campaña y para este cliente (numérica, incluye el último contacto)
- poutcome - Resultado de la campaña de marketing anterior (categórica; 'failure','nonexistent','success')
- y - El cliente ha contratado el depósito (binaria, yes, no)

Recuerda, si tuvieras que programar una función, comenta los argumentos de entrada y salida. **Explica el orden que estás siguiendo a la hora de elegir las visualizaciones y comenta las conclusiones que vas sacando.**


**(*) IMPORTANTE**: Puedes elegir realizar un modelo de clasificación y realizar visualizaciones en torno a ese modelo en los primeros dos apartados. Esta parte no es obligatoria. El objetivo de la clasificación sería predecir si el cliente se suscribirá a un depósito bancario (variable y).








### **Librerías necesarias**

In [1]:
# * Librerias estandar: 
import pandas as pd                 # Para la manipulación de los datasets
import numpy as np                  # Para operaciones matematicas
import sklearn.datasets             # Para los datasets

import plotly.express as px         # Plotly
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pickle                       # Esto sirve para el K-Means Clustering y poder guardar el modelo en el formator correcto. 
from plotly.subplots import make_subplots

# * Librerias de Dash
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output, State

# * Librerías Machine Learning
# * Modelos Existentes (Regresión Lineal, Logistica, Random Forest, K-Means Clustering, SVM)

from sklearn.model_selection import train_test_split 
# ? - Regresión Lineal + Logisticas
from sklearn.linear_model import LinearRegression, SGDClassifier, LogisticRegression
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.svm import SVC
# T-test
from scipy.stats import ttest_ind

# ? - Random Forest (+ Escalamiento necesario)
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# ? - Metricas de Precisión
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error,
    classification_report,
    confusion_matrix,
    silhouette_score
)

from sklearn.pipeline import Pipeline # ? - Para generar modelos de machine learning con escalamiento automatico

El objetivo del análisis es buscar patrones para entender mejor que tipo de perfil tienen los clientes que han contratado el depósito para buscar en su base de datos otros clientes parecidos para aumentar la respuesta y el ROI de futuras campañas de marketing directo intentando vender el mismo depósito. Por lo tanto, se pide:

1.  Realizar un análisis descriptivo de los datos con al menos, 6 visualizaciones diferentes. (3 Puntos) (*)

In [8]:
dataset = pd.read_csv("bank-full.csv", delimiter=';')
df = pd.DataFrame(dataset)

#Infd del dataset 
df.head()
df.info()


# Repaso de si hay missing values
df.isna().sum() # ? - No hay valores nulos 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [12]:
# ! - Objetivo del analisis es buscar patrones para entender el tipo de 
# !- perfil que han contratado el deposito. Es decir que la columna y = "yes"
# ROI - Futuras Campañas de marketing

# * Filtrado inicial del dataset 
df_deposito = df[df['y'] == 'yes']
df_no_deposito = df[df['y'] == 'no']

df_deposito.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
83,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
86,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
87,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
129,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
168,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


# Ejercicio 1: 6 gráficas 

Una historia de 3 partes:  (3 gráficas de cada)
* El perfil personal 
* El perfil financiero 

## El perfil personal

In [62]:
# 1. Distribución de edad
# Crear dos histogramas lado a lado para comparar la distribución de edad de los clientes que contrataron el depósito y los que no.
fig = make_subplots(rows = 1, cols = 1)
# Violin plot para ver la distribución de edad
fig.add_trace(go.Violin(x = df_deposito['age'], name = 'Deposito'), row = 1, col = 1)
fig.add_trace(go.Violin(x = df_no_deposito['age'], name = 'No Deposito'), row = 1, col = 1)
fig.update_layout(title = 'Distribución de Edad')
fig.show()

# Manera de combinar tanto el histograma como el box plot

^^Esta gráfica desea ver la comparación de la distribución de edades entre aquellos que decidieron no hacer un deposito y si hacer un deposito. Podemos observar que la distribución es bastance parecida, pero hay un crecimiento mas uniforme para los que si han cogido un deposito entre los 20 y 30 años.  

In [27]:
# 2. Diagrama sunburst para observar la distribución de los tepositos por educacion y trabajo
# ! - Diagrama Sunburst
# ? - Debemos de cambiar la columna y a 1 y 0 para poder hacer el diagrama
df_deposito['y'] = df_deposito['y'].apply(lambda x: 1)
df_no_deposito['y'] = df_no_deposito['y'].apply(lambda x: 0)

fig = px.sunburst(df_deposito, path=['education', 'job'], values='y')
fig.update_layout(
    title_text='Distribución de miembros de depositos por educación y trabajo',
)
fig.show()

^^Aqui podemos observar que tipo de perfiles son los más probables a hacer un deposito en proporción a su nivel de educación. Si se desea ver adentro del grupo de educación, cuales son los que más tendencia tienen a pagar, se pueden ver haciendo clic sobre el grupo educativo. 

* ESO - Tecnico
* Terciario - Managers
* Primaria - Obreros/Retriados
* Los demas - Managers

In [72]:
# 2 Bar Charts que muestran la proporcion de marital status entre los que contrataron el deposito y los que no
# ! - Bar Charts
# Group by education
df_edu = df_deposito.groupby('education').count().reset_index()
df_edu_n = df_no_deposito.groupby('education').count().reset_index()   

fig = make_subplots(rows = 1, cols = 2)

fig.add_trace(go.Bar(x = df_edu['education'], y = df_edu['balance'], name = 'Deposito'), row = 1, col = 1)
fig.add_trace(go.Bar(x = df_edu_n['education'], y = df_edu_n['balance'], name = 'No Deposito'), row = 1, col = 2)
fig.update_layout(title = 'Distribución de Estado Civil')
fig.show()

^^Distribución para ver que tipos de perfiles son los que más dinero ganan. 

## Perfil financiero

In [34]:
# 1. Generar pie charts para ver entre los que sí y no, los que tienen housing loan y los que no 
fig = make_subplots(rows = 1, cols = 2, specs = [[{'type':'domain'}, {'type':'domain'}]])
# Pie plot para ver los housing loan
fig.add_trace(go.Pie(labels = df_deposito['housing'].value_counts().index, values = df_deposito['housing'].value_counts().values, name = 'Deposito', title="Deposito"), row = 1, col = 1)
fig.add_trace(go.Pie(labels = df_no_deposito['housing'].value_counts().index, values = df_no_deposito['housing'].value_counts().values, name = 'No Deposito', title = "No Deposito"), row = 1, col = 2)
fig.update_layout(title = 'Percentage of Housing Loan')
fig.show()

^^Quería observar adentro de aquellos que hacían o no el deposito, que porcentaje tenía una hipoteca hecha u no. Podemos ver que para los que si hacen un deposito, la mayoría no tienen hipoteca, mientras que ocurre lo contrario para los otros. 

In [35]:
# 2. Grafica para comparar las edades con el balance de los que sí y no contrataron el depósito
fig = make_subplots(rows = 1, cols = 2)
fig.add_trace(go.Scatter(x = df_deposito['age'], y = df_deposito['balance'], mode = 'markers', name = 'Deposito'), row = 1, col = 1)
fig.add_trace(go.Scatter(x = df_no_deposito['age'], y = df_no_deposito['balance'], mode = 'markers', name = 'No Deposito'), row = 1, col = 2)
fig.update_layout(title = 'Balance vs Age')
fig.show()

^^Me interesaba ver la comparación entre edad y sueldo en un scatter plot, para ver aquellos que son mas probables de invertir en la oferta que les hacemos. Podemos observar que para los nivels superiores a 15k, perdemos mucha mas clientela. 

In [49]:
# Stacked bar chart para ver entre los que si y los que no, de aquellos que tienen un loan cuales tambien han hecho default 

# Stacked Bar Chart
fig = make_subplots(rows=1, cols=2)

# Obtener los datos para el gráfico
loan_default = df_deposito[(df_deposito['loan'] == 'yes') & (df_deposito['default'] == 'yes')]
loan_no_default = df_deposito[(df_deposito['loan'] == 'yes') & (df_deposito['default'] == 'no')]

no_loan_default = df_deposito[(df_deposito['loan'] == 'no') & (df_deposito['default'] == 'yes')]
no_loan_no_default = df_deposito[(df_deposito['loan'] == 'no') & (df_deposito['default'] == 'no')]

# Calcular los porcentajes
total_loan = len(loan_default) + len(loan_no_default)
total_no_loan = len(no_loan_default) + len(no_loan_no_default)

loan_default_percentage = len(loan_default) / total_loan * 100
loan_no_default_percentage = len(loan_no_default) / total_loan * 100

no_loan_default_percentage = len(no_loan_default) / total_no_loan * 100
no_loan_no_default_percentage = len(no_loan_no_default) / total_no_loan * 100

# Agregar las barras apiladas al gráfico con los porcentajes
fig.add_trace(go.Bar(x=['Loan'], y=[loan_default_percentage], name='Default', marker_color='red', text=f'{loan_default_percentage:.2f}%'), row=1, col=1)
fig.add_trace(go.Bar(x=['Loan'], y=[loan_no_default_percentage], name='No Default', marker_color='green', text=f'{loan_no_default_percentage:.2f}%'), row=1, col=1)

fig.add_trace(go.Bar(x=['No Loan'], y=[no_loan_default_percentage], name='Default', marker_color='red', text=f'{no_loan_default_percentage:.2f}%'), row=1, col=2)
fig.add_trace(go.Bar(x=['No Loan'], y=[no_loan_no_default_percentage], name='No Default', marker_color='green', text=f'{no_loan_no_default_percentage:.2f}%'), row=1, col=2)

# Personalizar el diseño del gráfico
fig.update_layout(title='Stacked Bar Chart - Loan and Default for those with a deposit',
                  yaxis_title='Count',
                  barmode='stack')

# Mostrar el gráfico
fig.show()


^^Aquí quise observar de la proporción que si que había invertido en nuestro deposito, que proporción estaba tomando un prestamo adicional actualmente, y que proporción de ellos no habían sido capaz de cumplirlo. 

In [65]:
# Matriz de correlación para ver la relación entre las variables
# Antes debo de convertir las variables categóricas a numéricas
df['y'] = pd.Categorical(df['y']).codes
df['default'] = pd.Categorical(df['default']).codes
df['housing'] = pd.Categorical(df['housing']).codes
df['loan'] = pd.Categorical(df['loan']).codes
df['job'] = pd.Categorical(df['job']).codes
df['education'] = pd.Categorical(df['education']).codes
df['marital'] = pd.Categorical(df['marital']).codes
df['contact'] = pd.Categorical(df['contact']).codes
df['month'] = pd.Categorical(df['month']).codes
df['poutcome'] = pd.Categorical(df['poutcome']).codes

corr = df.corr(numeric_only=True)
figure = ff.create_annotated_heatmap(
        x = list(corr.columns),
        y = list(corr.index),
        z = np.array(corr),
        annotation_text = np.round(np.array(corr)*100),
        hoverinfo='z',
        colorscale=px.colors.sequential.GnBu
)

figure.show()

## Modelo ML ? 

# Conclusiones: 

3. Concluir todo este análisis haciendo recomendaciones para la mejora de futuras campañas de contacto directo a partir de los resultados obtenidos de los análisis realizados con los datos. (2 Puntos)

Como podemos observar en las gráficas de arriba, tenemos que la media edad de aquellos que tienen el deposito es de alrededor de 30 años, con los que tienen una educación secundaria siendo aquellos que mas nos suelen comprar. Las profesiones a las que más les atraemos son a aquellas posiciones que son: 

* Técnicos
* Managers 
* Blue-collar workers/Mano de obra. 

Por parte de la distribución de edades y precios, tambien podemos observar en el scatter plot que aquellos cuyos salarios sobrepasen los 10k euros, no suelen quedarse a la hora de ofrecerles una oportunidad, por lo tanto, debemos hacer target a aquellos más pequeños. 

En la grafica de barras, tambien se puede observar que de los que si que toman nuestros prestamos, existe una proporción muy baja de defaults, por lo tanto nos podemos asegurar que el dinero que si que dejamos a nuestros clientes tienen una probabilidad muy alta de ser devuelto y que podamos obtener nuestros intereses de vuelta. 
